# AI Config AI Metrics - Cookbook

Prerequisites: `LAUNCHDARKLY_SDK_KEY`, `LAUNCHDARKLY_API_TOKEN`, AI Config created (see `aiconfig-create`)

In [1]:
%pip install launchdarkly-server-sdk launchdarkly-server-sdk-ai openai requests python-dotenv -q

import os
from pathlib import Path
from dotenv import load_dotenv

def find_repo_root(start_path: Path = None) -> Path:
    current = start_path or Path.cwd()
    for parent in [current] + list(current.parents):
        if (parent / '.git').exists():
            return parent
    return current

repo_root = find_repo_root()
load_dotenv(repo_root / '.env')
print(f"[OK] Loaded environment from {repo_root / '.env'}")


[notice] A new release of pip is available: 25.1.1 -> 26.0
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
[OK] Loaded environment from /Users/ld_scarlett/Documents/Github/agent-skills/.env


In [2]:
# SDK initialization (see aiconfig-sdk for details)
from ldclient import Context
from ldai.client import LDAIClient, AICompletionConfigDefault
import ldclient
from ldclient.config import Config

SDK_KEY = os.environ.get("LAUNCHDARKLY_SDK_KEY")
ldclient.set_config(Config(SDK_KEY))
ld_client = ldclient.get()
ai_client = LDAIClient(ld_client)

print(f"[OK] SDK initialized: {ld_client.is_initialized()}")

# Get config for testing tracker methods
context = Context.builder("cookbook-test-user").build()
config = ai_client.completion_config(
    "content-assistant",
    context,
    AICompletionConfigDefault(enabled=False),
    {}
)
print(f"[OK] Got config: enabled={config.enabled}, model={config.model.name if config.model else 'N/A'}")

[OK] SDK initialized: True
[OK] Got config: enabled=True, model=gpt-4


---
## Test Tracker Methods

In [3]:
from ldai.tracker import TokenUsage
import time

tracker = config.tracker
print(f"[OK] Got tracker: {type(tracker).__name__}")

# Test track_success()
tracker.track_success()
print(f"[OK] track_success()")

# Test track_error()
tracker.track_error()
print(f"[OK] track_error()")

# Test track_tokens() with TokenUsage
tokens = TokenUsage(total=30, input=10, output=20)
tracker.track_tokens(tokens)
print(f"[OK] track_tokens(TokenUsage(total=30, input=10, output=20))")

# Test track_duration()
tracker.track_duration(1.5)
print(f"[OK] track_duration(1.5)")

# Test track_time_to_first_token()
tracker.track_time_to_first_token(0.25)
print(f"[OK] track_time_to_first_token(0.25)")

# Test track_duration_of()
def slow_fn():
    time.sleep(0.1)
    return "result"

result = tracker.track_duration_of(slow_fn)
print(f"[OK] track_duration_of(fn) -> {result}")

ld_client.flush()
print(f"[OK] Flushed metrics")

[OK] Got tracker: LDAIConfigTracker
[OK] track_success()
[OK] track_error()
[OK] track_tokens(TokenUsage(total=30, input=10, output=20))
[OK] track_duration(1.5)
[OK] track_time_to_first_token(0.25)
[OK] track_duration_of(fn) -> result
[OK] Flushed metrics


---
## OpenAI Automatic Tracking
From: `SKILL.md` lines 46-68

In [4]:
import openai
from ldai.tracker import TokenUsage

def track_openai_completion(config, prompt: str):
    """Track OpenAI completion with automatic metrics."""
    if not config.enabled:
        return None

    tracker = config.tracker

    # Wrap OpenAI call - automatically captures tokens, duration, success/failure
    response = tracker.track_openai_metrics(
        lambda: openai.chat.completions.create(
            model=config.model.name,
            messages=[
                {"role": "system", "content": config.messages[0].content},
                {"role": "user", "content": prompt}
            ]
        )
    )

    return response.choices[0].message.content

print("[OK] track_openai_completion() defined")

[OK] track_openai_completion() defined


In [5]:
# Test track_openai_completion
print("=== Testing track_openai_completion ===")

openai_key = os.environ.get("OPENAI_API_KEY")
if openai_key and config.enabled:
    openai.api_key = openai_key
    result = track_openai_completion(config, "Say hello in one word")
    if result:
        print(f"[OK] Got response: {result[:50]}...")
        ld_client.flush()
else:
    print("[INFO] OPENAI_API_KEY not set or config disabled - skipping live test")
    print("[OK] Function defined and ready for use")

=== Testing track_openai_completion ===
[OK] Got response: Hi...


---
## Manual Token Tracking
From: `SKILL.md` lines 75-107

In [6]:
from ldai.tracker import TokenUsage

def track_anthropic_completion(config, prompt: str):
    """Track Anthropic completion with manual metrics."""
    import anthropic
    client = anthropic.Anthropic()

    if not config.enabled:
        return None

    tracker = config.tracker

    # Track duration of the call
    response = tracker.track_duration_of(
        lambda: client.messages.create(
            model=config.model.name,
            max_tokens=1024,
            messages=[{"role": "user", "content": prompt}]
        )
    )

    # Manually track tokens using TokenUsage object
    if hasattr(response, 'usage'):
        tokens = TokenUsage(
            total=response.usage.input_tokens + response.usage.output_tokens,
            input=response.usage.input_tokens,
            output=response.usage.output_tokens
        )
        tracker.track_tokens(tokens)

    tracker.track_success()
    return response.content[0].text

print("[OK] track_anthropic_completion() defined")

[OK] track_anthropic_completion() defined


---
## Streaming Metrics
From: `SKILL.md` lines 112-156

In [7]:
import time
from ldai.tracker import TokenUsage

def track_streaming_completion(config, prompt: str):
    """Track metrics for streaming responses."""
    import openai

    if not config.enabled:
        return None

    tracker = config.tracker
    start_time = time.time()
    first_token_time = None

    stream = openai.chat.completions.create(
        model=config.model.name,
        messages=[{"role": "user", "content": prompt}],
        stream=True
    )

    response_text = ""
    for chunk in stream:
        if first_token_time is None and chunk.choices[0].delta.content:
            first_token_time = time.time()
            tracker.track_time_to_first_token(first_token_time - start_time)

        if chunk.choices[0].delta.content:
            response_text += chunk.choices[0].delta.content

    # Track final metrics
    tracker.track_duration(time.time() - start_time)
    tracker.track_success()

    # Estimate tokens (or use tiktoken for accuracy)
    estimated_input = len(prompt.split()) * 2
    estimated_output = len(response_text.split()) * 2
    tokens = TokenUsage(
        total=estimated_input + estimated_output,
        input=estimated_input,
        output=estimated_output
    )
    tracker.track_tokens(tokens)

    return response_text

print("[OK] track_streaming_completion() defined")

[OK] track_streaming_completion() defined


---
## Retrieving Metrics via API
From: `SKILL.md` lines 187-225

In [9]:
import requests
import time
import os

def get_ai_metrics(project_key: str, config_key: str, hours: int = 24):
    """Get AI metrics for the last N hours."""
    API_TOKEN = os.environ.get("LAUNCHDARKLY_API_TOKEN")

    now = int(time.time())
    start = now - (hours * 3600)

    url = f"https://app.launchdarkly.com/api/v2/projects/{project_key}/ai-configs/{config_key}/metrics"

    params = {
        "from": start,
        "to": now,
        "env": "production"
    }

    headers = {
        "Authorization": API_TOKEN,
        "LD-API-Version": "beta"
    }

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        metrics = response.json()
        print(f"[INFO] Metrics for {config_key} (last {hours} hours):")
        print(f"   Total Tokens: {metrics.get('totalTokens', 0):,}")
        print(f"   Total Cost: ${metrics.get('totalCost', 0):.4f}")
        print(f"   Request Count: {metrics.get('requestCount', 0):,}")
        print(f"   Avg Latency: {metrics.get('avgLatency', 0):.2f}s")
        print(f"   Success Rate: {metrics.get('successRate', 0):.1f}%")
        return metrics
    else:
        print(f"[ERROR] Failed to get metrics: {response.status_code}")
        return None

print("[OK] get_ai_metrics() defined")

[OK] get_ai_metrics() defined


In [10]:
# Test get_ai_metrics
print("=== Testing get_ai_metrics ===")
metrics = get_ai_metrics("support-ai", "content-assistant", hours=24)
if metrics:
    print("[OK] Metrics retrieved successfully")
else:
    print("[INFO] No metrics available (endpoint may not be active yet)")

=== Testing get_ai_metrics ===
[INFO] Metrics for content-assistant (last 24 hours):
   Total Tokens: 0
   Total Cost: $0.0000
   Request Count: 0
   Avg Latency: 0.00s
   Success Rate: 0.0%
[OK] Metrics retrieved successfully
